In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx to Coorti 2016-2018 estrazione 2020 02 21 per AI (1).xlsx
Saving esami coorti 2016-2018 - 2020 02 21 per AI.xlsx to esami coorti 2016-2018 - 2020 02 21 per AI (1).xlsx
User uploaded file "Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx" with length 6481955 bytes
User uploaded file "esami coorti 2016-2018 - 2020 02 21 per AI.xlsx" with length 15391182 bytes


In [ ]:
import pandas as pd
import numpy as np

#Read dataset
features =  pd.read_excel("Coorti 2016-2018 estrazione 2020 02 21 per AI.xlsx")
esami = pd.read_excel("esami coorti 2016-2018 - 2020 02 21 per AI.xlsx")

Siamo interessati al totale dei CFU ottenuti e la media dei voti ottenuti

In [ ]:
esami = esami.drop('Cod.Orientamento', axis=1)
esami = esami.drop('Cod.Indirizzo', axis=1)
esami = esami.drop('Cod.Materia', axis=1)
esami = esami.drop('Materia', axis=1)
esami = esami.drop('Giorno_esame', axis=1)

Calcoliamo la somma dei CFU per ogni studente la mergiamo con i dati di ogni studente e i valori NaN dei CFU verranno rimpiazzati con 0

In [ ]:
tot_cfu = esami.groupby('ID_Studente')['CFUsuperati'].sum()
features = features.merge(tot_cfu,how='outer', on='ID_Studente')
features['CFUsuperati'] = features['CFUsuperati'].fillna(0)

Calcoliamo la media di voti per ogni studente e la mergiamo con i dati di ogni studente e i valori NaN dei voti verranno rimpiazzati con 0

In [ ]:
esami = esami.drop('CFUsuperati', axis=1)
esami = esami.dropna()
media_voti = esami.groupby('ID_Studente')['Voto_se_numerico'].mean()
features = features.merge(media_voti,how='outer', on='ID_Studente')
features['Voto_se_numerico'] = features['Voto_se_numerico'].fillna(0)

Rimpiazziamo i codici relativi a determinate feature con i nomi delle feature, così da poter utilizzare la funzione get_dummies() ed ottenere un one-hot-encoding

In [ ]:
features["Diploma_scuola_superiore"] = features["Diploma_scuola_superiore"].map({1: "Classico", 2: "Scientifico", 3: "Linguistico", 4: "Magistrale", 5: "Artistico", 6: "Tecnico", 7: "Professionale", 8: "Altro_italiano", 9: "Estero", 99: "Non_disponibile"})
features["area_geografica_scuolasuperiore"] = features["area_geografica_scuolasuperiore"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["area_geografica_residenza"] = features["area_geografica_residenza"].map({1: "Emilia_romagna", 2: "Nord", 3: "Centro", 4: "Sud_isole", 5: "Estero", 99: "Non_disponibile"})
features["Ambito"] = features["Ambito"].map({1: "Economia", 2: "Farmacia", 3: "Giurisprudenza", 4: "Ingegneria", 5: "Lingue", 6: "Medicina", 7: "Veterinaria", 8: "Psicologia", 9: "Scienze", 10: "Scienze_agroalimanetari", 11: "Scienze_educazione_formazione", 12: "Scienze_motorie", 13: "Scienze_politiche", 14: "Scienze_statistiche", 15: "Sociologia", 16: "Studi_umanistici"})
features = pd.get_dummies(features)

Create and add age in DataFrame

In [ ]:
from datetime import datetime, date

def getAge(dataNascita, coorte):
    ts = pd.to_datetime(str(dataNascita)) 
    born = ts.strftime('%d/%m/%Y')
    born = datetime.strptime(born, "%d/%m/%Y").date()
    date = "31/12/"+str(coorte)
    dataAttuale = datetime.strptime(date, "%d/%m/%Y").date()
    age = dataAttuale.year - born.year - ((dataAttuale.month, dataAttuale.day) < (born.month, born.day))
    return age

#add new column with NaN
features["Eta"] = np.nan

for ind in features.index:
    studente = features.loc[ind]
    dataNascita = studente["DataNascita"]
    coorte = studente["Coorte"]
    #fill the new column with age 
    features.at[ind,'Eta'] = getAge(dataNascita, coorte)

print(features)

       ID_Studente  Coorte DataNascita  Genere  voto_scuola_superiore  \
0                1    2017  1998-02-26       1                   79.0   
1                2    2016  1997-10-06       1                   75.0   
2                3    2017  1986-08-29       1                   74.0   
3                4    2017  1998-01-23       1                   63.0   
4                5    2018  1999-07-04       2                   98.0   
...            ...     ...         ...     ...                    ...   
67275        67276    2016  1997-04-22       1                   83.0   
67276        67277    2017  1998-01-08       1                  100.0   
67277        67278    2016  1997-04-16       2                   75.0   
67278        67279    2017  1995-10-28       2                    NaN   
67279        67280    2016  1992-09-15       1                   65.0   

       Classe_ISEE  Merito_ISEE  OFA_assegnati  OFA_superati   CdS  ...  \
0                5     36332.49              0  

Abbiamo sostituito i valori NaN della feature "voto_scuola_superiore" con la media. Inoltre sono state droppate le colonne non utili ai fini dell'addestramento, ed infine eliminate le tuple contente almeno un NaN

In [ ]:
mean_value = features['voto_scuola_superiore'].mean()
features['voto_scuola_superiore'] = features['voto_scuola_superiore'].fillna(mean_value)
features = features.drop('ID_Studente', axis = 1)
features = features.drop('Classe_ISEE', axis = 1)
features = features.drop('DataNascita', axis = 1)
features = features.drop('Coorte', axis = 1)
features = features.dropna()

Abbiamo bilanciato il dataset con approccio undersampling pareggiando il numero di dropout e non droput

In [ ]:
#BILANCIAMENTO 1
#Data preprocessing
dropout = pd.DataFrame()
noDropout = pd.DataFrame()

for ind in features.index:
    studente = pd.DataFrame(features.loc[[ind]])
    if features["Abbandoni"][ind] == 1:
        dropout = pd.concat([studente, dropout], ignore_index=True, axis = 0)
    else:
        noDropout = pd.concat([studente, noDropout], ignore_index=True, axis = 0)

#Bilanciamento dataset
final = pd.DataFrame()
for ind in dropout.index:
    studente_drop = pd.DataFrame(dropout.loc[[ind]])
    studente_noDrop = pd.DataFrame(noDropout.loc[[ind]])
    final = pd.concat([studente_drop, final], ignore_index=True, axis = 0)
    final = pd.concat([studente_noDrop, final], ignore_index=True, axis = 0)

features = final
print(len(features))

4186


La seguente cella serve a prescindere dal tipo di bilanciamento 

In [ ]:
#Dividing the data into attributes and labels
X = features.drop('Abbandoni', axis=1)
y = features['Abbandoni']

In [ ]:
#BILANCIAMENTO 2
"""
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

#Resampling data
rus = RandomUnderSampler(sampling_strategy = "majority", random_state=42)
X_res, y_res = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

X = X_res
y = y_res
"""

'\nfrom collections import Counter\nfrom sklearn.datasets import make_classification\nfrom imblearn.under_sampling import RandomUnderSampler\n\n#Resampling data\nrus = RandomUnderSampler(sampling_strategy = "majority", random_state=42)\nX_res, y_res = rus.fit_resample(X, y)\nprint(\'Resampled dataset shape %s\' % Counter(y_res))\n\nX = X_res\ny = y_res\n'

In [ ]:
from sklearn.model_selection import train_test_split as tts
#Dividing data into training and test sets
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.20, random_state=42)

Creo il modello SVM

In [ ]:
from sklearn import svm
#Creating Support Vector Machine Model
print("Creating SVM Model\n")
svclassifier = svm.SVC(C=0.01, kernel='linear')

Creating SVM Model



Avviamo l'addestramento del modello

In [ ]:
#Training SVM Model
print("Training SVM Model\n")
svclassifier.fit(X_train, y_train)

Training SVM Model



SVC(C=0.01, kernel='linear')

Eseguiamo il modello con i set del test per verificare le predizioni

In [ ]:
#Making prediction
print("Making prediction\n")
y_pred = svclassifier.predict(X_test)

Making prediction



Stampa dei risultati

In [ ]:
from sklearn import metrics

#Evaluating the Algorithm
print("Evaluating the Algorithm:\n")
tn, fp, fn, tp = metrics.confusion_matrix(y_test,y_pred).ravel()
print("true negative: ", tn)
print("false negative: ", fn)
print("true positive: ", tp)
print("false positive: ", fp)
print(metrics.classification_report(y_test,y_pred))

Evaluating the Algorithm:

true negative:  393
false negative:  42
true positive:  359
false positive:  44
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       437
           1       0.89      0.90      0.89       401

    accuracy                           0.90       838
   macro avg       0.90      0.90      0.90       838
weighted avg       0.90      0.90      0.90       838



In [ ]:
#Evaluating Prediction Accuracy
print("Evaluating Prediction Accuracy:\n")
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Evaluating Prediction Accuracy:

Accuracy: 0.8973747016706444
